In [1]:
pip install ipykernel

Note: you may need to restart the kernel to use updated packages.


In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
import json
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service

s=Service("/Users/thanuathitya/Scrapping/chromedriver-mac-x64/chromedriver")
driver=webdriver.Chrome(service=s)

# Login
def login():
    with open('login.txt') as login_file:
        lines = login_file.readlines()

    email = lines[0].strip()
    password = lines[1].strip()

    driver.get("https://www.linkedin.com/login")

    email_field = driver.find_element(By.ID, "username")
    email_field.send_keys(email)
    
    password_field = driver.find_element(By.ID, "password")
    password_field.send_keys(password)

    login_button = driver.find_element(By.XPATH, "//*[@id=\"organic-div\"]/form/div[3]/button")
    login_button.click()

    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id=\"global-nav\"]"))
        )
    except TimeoutException:
        print("Login failed or took too long. Please check your credentials and network connection.")
        driver.quit()

# Return all profiles URLs of M&A employees of a certain company
def getProfileURLs(companyName):
    driver.get(f"https://www.linkedin.com/company/{companyName}/people/?keywords=M%26A%2CMergers%2CAcquisitions")
    time.sleep(3)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

    source = BeautifulSoup(driver.page_source, "html.parser")

    visibleEmployeesList = []
    visibleEmployees = source.find_all('a', class_='app-aware-link')
    for profile in visibleEmployees:
        href = profile.get('href')
        if '/in/' in href:
            visibleEmployeesList.append(href)

    invisibleEmployeeList = []
    invisibleEmployees = source.find_all('div', class_='artdeco-entity-lockup artdeco-entity-lockup--stacked-center artdeco-entity-lockup--size-7 ember-view')
    for invisible_guy in invisibleEmployees:
        title = invisible_guy.find('div', class_='lt-line-clamp lt-line-clamp--multi-line ember-view').get_text(strip=True)
        invisibleEmployeeList.append(title)
        profile_pic_link = invisible_guy.find('img', class_='lazy-image ember-view')['src']
        invisibleEmployeeList.append(profile_pic_link)

    return visibleEmployeesList, invisibleEmployeeList

# Parses a type 2 job row
def parseType2Jobs(alltext):
    job_groups = []
    company = alltext[16].strip()
    total_duration = alltext[20].strip()

    num_jobs = [(count, index) for index, count in enumerate(alltext) if count == '' or count == ' ']

    for count, index in num_jobs:
        if 'time' in alltext[index + 5].lower().split('-'):
            job_groups.append((alltext[index].strip(), alltext[index + 8].strip()))
        else:
            job_groups.append((alltext[index].strip(), alltext[index + 4].strip()))
    return 'type2job', company, total_duration, job_groups

# Parses a type 1 job row
def parseType1Job(alltext):
    job_title = alltext[16].strip()
    company = alltext[20].strip()
    duration = alltext[23].strip()
    return 'type1job', job_title, company, duration

# Returns LinkedIn profile information
def returnProfileInfo(employeeLink, companyName):
    driver.get(employeeLink)
    time.sleep(2)
    source = BeautifulSoup(driver.page_source, "html.parser")

    profile = [companyName]
    info = source.find('div', class_='mt2 relative')
    name = info.find('h1', class_='text-heading-xlarge inline t-24 v-align-middle break-words').get_text(strip=True)
    title = info.find('div', class_='text-body-medium break-words').get_text(strip=True)
    profile.append(name)
    profile.append(title)

    time.sleep(1)
    experiences = source.find_all('li', class_='artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column')

    for x in experiences[1:]:
        alltext = x.getText().split('\n')
        start_identifier = next((index for index, value in enumerate(alltext) if value.strip()), len(alltext))

        if start_identifier == 16:
            if 'university' in alltext[16].lower() or 'college' in alltext[16].lower():
                profile.append(('education', alltext[16].strip(), alltext[20].strip()))
            elif 'issued' in alltext[23].lower():
                profile.append(('certification', alltext[16].strip(), alltext[20].strip()))
        elif start_identifier == 12:
            if not alltext[16].strip() and len(alltext) > 24:
                profile.append(('skill', alltext[12].strip()))

    driver.get(driver.current_url + '/details/experience/')
    time.sleep(2)
    source = BeautifulSoup(driver.page_source, "html.parser")
    exp = source.find_all('li')

    for e in exp[13:]:
        row = e.getText().split('\n')
        if row[:16] == ['', '', '', '', '', '', ' ', '', '', '', '', '', '', '', '', '']:
            if 'yrs' in row[20].split(' '):
                profile.append(parseType2Jobs(row))
            else:
                profile.append(parseType1Job(row))

    return profile

if __name__ == "__main__":
    companies = ['apple'] # Add more company names as needed
    login()
    employees = {}
    for company in companies:
        visible_profiles, invisible_profiles = getProfileURLs(company)
        for employee in visible_profiles:
            employees[employee] = returnProfileInfo(employee, company)
    with open('m&a.json', 'w') as f:
        json.dump(employees, f)
    time.sleep(10)
    driver.quit()


Login failed or took too long. Please check your credentials and network connection.


MaxRetryError: HTTPConnectionPool(host='localhost', port=49901): Max retries exceeded with url: /session/bfab24032f779883c5c7317b10c63007/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1236005c0>: Failed to establish a new connection: [Errno 61] Connection refused'))